In [1]:
import pandas as pd
import numpy as np
from enum import IntEnum

In [2]:
class Code(IntEnum):
    SHOT = 1 # AI agent kills enemy
    RIGHT_TO_LEFT = 2 # moves from right to left side
    LEFT_TO_RIGHT = 3 # moves from left to right side
    AI_DEATH = 4 # AI agent ship dies
    INACTIVE = 5 #not shooting
    REACTIVE = 6 # AI agent shoots again after not shooting for a while
    AI_FINISH = 7 # AI side enemies finished
    P_FINISH = 8 # Player side enemies finished
    FINISH = 9 # both sides finished
    TESTING = 10 # player tested out (-> 999)
    MISTAKE = 11 # pressed by mistake (-> 999)
    R_ENEMIES_LOW = 12 # Right side is low and AI agent is on that side
    IGNORANT_LOW = 14 # AI agent and Player agent both on left, Right side enemies are low
    P_DEATH = 15 # Player ship dies
    IGNORANT_MORE = 18 # AI agent and Player agent both on left, Right side has more enemies that left
    UNKNOWN = 999

In [3]:
def str_to_array(s):
    """ convert '['1,2,3']' to [1,2,3] """
    no_brackets = s[1:-1]
    return list(map(int, np.array(no_brackets.split(','))))

In [4]:
df_codes = pd.read_csv("pilot3_data/bucket_results/buckets_160.csv")

df = pd.DataFrame(columns = ['player_id', 'mode', 'frame_number', 'signal_type', 'num_code', 'key'])

# clean up
df_codes.loc[df_codes['num_code'] == '[]', ['num_code']] = '[999]'
# df_codes['code'] = df_codes['code'].map(str_to_array)
df_codes['num_code'] = df_codes['num_code'].map(str_to_array)

for index, row in df_codes.iterrows():
    for x in row['num_code']:
        df = df.append({'player_id': row['player_id'],
                        'mode': row['mode'],
                        'frame_number': row['frame_number'],
                        'signal_type': row['signal_type'],
                        'num_code': x,
                        'key': x},
                        ignore_index=True)
        

In [5]:
# man_codes = pd.read_csv("my_buckets.csv")[['player_id', 'mode', 'frame_number', 'signal_type', 'code']]
# df_man = pd.DataFrame(columns = ['player_id', 'mode', 'frame_number', 'signal_type', 'man_code', 'key'])

# def clean_man_codes(a):
#     if 10 in a or 11 in a:
#         if 10 in a:
#             a.remove(10)
#         if 11 in a:
#             a.remove(11)
#         a.append(999)
#     return a

# # clean up
# man_codes['code'] = man_codes['code'].map(str_to_array)

# man_codes['code'] = man_codes['code'].map(clean_man_codes)

# for index, row in man_codes.iterrows():
#     for x in row['code']:
#         df_man = df_man.append({'player_id': row['player_id'],
#                         'mode': row['mode'],
#                         'frame_number': row['frame_number'],
#                         'signal_type': row['signal_type'],
#                         'man_code': x,
#                         'key': x},
#                         ignore_index=True)

In [6]:
# df_combined = pd.merge(df, df_man, how='outer', on=['player_id', 'mode', 'frame_number', 'signal_type', 'key'])
df_combined = df
df_combined.to_csv('pilot3_data/new.csv')

In [7]:
df_combined['num_code']

0        1
1        2
2       18
3        2
4        5
      ... 
328    999
329      2
330      7
331      2
332      1
Name: num_code, Length: 333, dtype: object

In [8]:
# df_combined['found_by'] = df_combined.apply(lambda x: 'man' if pd.isnull(x['num_code']) else ('predict' if pd.isnull(x['man_code']) else 'both'), axis=1)
df_combined





,player_id,mode,frame_number,signal_type,num_code,key
0,5fc9a415aacaf14df94e6868,1,371,up,1,1
1,5fc9a415aacaf14df94e6868,1,599,down,2,2
2,5fc9a415aacaf14df94e6868,1,599,down,18,18
3,5fc9a415aacaf14df94e6868,1,1211,down,2,2
4,5fc9a415aacaf14df94e6868,1,1211,down,5,5
...,...,...,...,...,...,...
328,610aebd03f7daffc2fcce4b9,2,1204,up,999,999
329,610aebd03f7daffc2fcce4b9,2,1952,up,2,2
330,610aebd03f7daffc2fcce4b9,2,1952,up,7,7
331,610aebd03f7daffc2fcce4b9,2,2186,up,2,2


In [9]:
# get 2d tables
df_up = df_combined.loc[df_combined['signal_type'] == 'up']
df_down = df_combined.loc[df_combined['signal_type'] == 'down']

df_up['key'] = df_up.key.apply(lambda x: Code(x).name)
df_down['key'] = df_down.key.apply(lambda x: Code(x).name)

cross_up = pd.crosstab(index=df_up['key'], columns=['predict'],
#                              columns=df_up["found_by"],
                             margins=True)

cross_down = pd.crosstab(index=df_down['key'], columns=['predict'],
#                              columns=df_down["found_by"],
                             margins=True)

/Users/yoony/.local/share/virtualenvs/error_recovery_si-OLjJtCo6/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/Users/yoony/.local/share/virtualenvs/error_recovery_si-OLjJtCo6/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [10]:
cross_up.to_csv('pilot3_data/up_signal_dist.csv')
cross_down.to_csv('pilot3_data/down_signal_dist.csv')

In [20]:
d = pd.read_csv('pilot3_data/down_signal_dist.csv').drop(columns=['All'])
# d = d.rename(columns={'man': 'manual'})

In [21]:
d['enum_val'] = d['key'].apply(lambda x: None if x == 'All' else getattr(Code, x).value)
d.sort_values(by=['enum_val'])


,key,predict,enum_val
5,RIGHT_TO_LEFT,9,2.0
0,AI_DEATH,11,4.0
3,INACTIVE,23,5.0
6,R_ENEMIES_LOW,10,12.0
1,IGNORANT_LOW,7,14.0
4,P_DEATH,10,15.0
2,IGNORANT_MORE,13,18.0
7,UNKNOWN,9,999.0
8,All,92,NaN


In [22]:
u = pd.read_csv('pilot3_data/up_signal_dist.csv').drop(columns=['All'])
# u = u.rename(columns={'man': 'manual'})

In [23]:
u['enum_val'] = u['key'].apply(lambda x: None if x == 'All' else getattr(Code, x).value)
u.sort_values(by=['enum_val'])


,key,predict,enum_val
5,SHOT,163,1.0
4,RIGHT_TO_LEFT,19,2.0
2,LEFT_TO_RIGHT,7,3.0
0,AI_FINISH,13,7.0
3,P_FINISH,13,8.0
1,FINISH,10,9.0
6,UNKNOWN,16,999.0
7,All,241,NaN
